In [28]:
import torch
import math
import numpy as np
import json
import pandas as pd
import glob

from src.constants import OUTPUTS_DIR, MODELS_DIR
from src.utils import enumerate_binary_inputs

In [2]:
experiment_file_1 = OUTPUTS_DIR / "train_turbo_table_bcjr_renorm_init.json"

exp_files = [experiment_file_1]

In [3]:
training_runs = []
validation_runs = []
for exp_file in exp_files:
    with open(exp_file, "r") as e:
        results = json.load(e)
    for d in results["data"]:
        if "results" not in d:
            continue
            
        training_results = pd.DataFrame([res for res in d["results"] if res["type"] == "training"])
        validation_results = pd.DataFrame([{**res, "step": (d["results"][i-1]["step"] + 1 if i > 0 else 0)} for i, res in enumerate(d["results"]) if res["type"] == "validation"])
        preambles = pd.DataFrame([{**d["preamble"]["args"], **d["preamble"]}])
        training_results = training_results.merge(preambles, how="cross")
        validation_results = validation_results.merge(preambles, how="cross")
        
        
        training_runs.append(training_results)
        validation_runs.append(validation_results)

training = pd.concat(training_runs, axis=0)
validation = pd.concat(validation_runs, axis=0)

arg_select_cols = ["adam_lr", "block_len", "snr", "init_method", "window","num_iter", "use_max", "renormalize_table", "argument_hash"]
training_select_cols = ["xe", "ber", "bler", "step"]
val_select_cols = ["xe__mean", "ber__mean", "bler__mean", "xe__err", "ber__err", "bler__err", "step", ]

training = training[training_select_cols + arg_select_cols]
validation = validation[val_select_cols + arg_select_cols]

# lr_filter = [0.01]
# training = training[training["adam_lr"].isin(lr_filter)]
# validation = validation[validation["adam_lr"].isin(lr_filter)]

display(training.head())
display(validation.head())

,xe,ber,bler,step,adam_lr,block_len,snr,init_method,window,num_iter,use_max,renormalize_table,argument_hash
0,0.000008,0.0,0.0,0,0.01,100,4.0,parity,5,5,True,True,c13051876a313e20eae5303c6373beffca78b3c8
1,0.000007,0.0,0.0,1,0.01,100,4.0,parity,5,5,True,True,c13051876a313e20eae5303c6373beffca78b3c8
2,0.000008,0.0,0.0,2,0.01,100,4.0,parity,5,5,True,True,c13051876a313e20eae5303c6373beffca78b3c8
3,0.000012,0.0,0.0,3,0.01,100,4.0,parity,5,5,True,True,c13051876a313e20eae5303c6373beffca78b3c8
4,0.000012,0.0,0.0,4,0.01,100,4.0,parity,5,5,True,True,c13051876a313e20eae5303c6373beffca78b3c8


,xe__mean,ber__mean,bler__mean,xe__err,ber__err,bler__err,step,adam_lr,block_len,snr,init_method,window,num_iter,use_max,renormalize_table,argument_hash
0,0.000802,0.000264,0.024609,0.000126,0.000034,0.003062,0,0.01,100,4.0,parity,5,5,True,True,c13051876a313e20eae5303c6373beffca78b3c8
1,0.000806,0.000258,0.024316,0.000131,0.000033,0.003044,50,0.01,100,4.0,parity,5,5,True,True,c13051876a313e20eae5303c6373beffca78b3c8
2,0.000723,0.000241,0.022949,0.000111,0.000032,0.002960,100,0.01,100,4.0,parity,5,5,True,True,c13051876a313e20eae5303c6373beffca78b3c8
3,0.000737,0.000259,0.023340,0.000094,0.000035,0.002984,150,0.01,100,4.0,parity,5,5,True,True,c13051876a313e20eae5303c6373beffca78b3c8
4,0.000739,0.000238,0.022266,0.000140,0.000033,0.002916,200,0.01,100,4.0,parity,5,5,True,True,c13051876a313e20eae5303c6373beffca78b3c8


In [10]:
arg_hashes = training["argument_hash"].unique()


In [12]:
from glob import glob
models = glob(str(MODELS_DIR / "*.pt"))
model_dict = {arg_hash: [f for f in models if arg_hash in f][0] for arg_hash in arg_hashes}
    

In [29]:
details = training.groupby(["argument_hash"]).agg("first")
def make_name(row):
    return f"block_len={row['block_len']}:lr={row['adam_lr']}:snr={row['snr']}:w={row['window']}:init={row['init_method']}:it={row['num_iter']}:max={row['use_max']}:r={row['renormalize_table']}"

for arg_hash, row in details.iterrows():
    name = make_name(row)
    model = model_dict[arg_hash]
    print("=====================")
    print(name)
    s_dict = torch.load(model)
    k1= "noninterleaved_encoder.table"
    k2 = "interleaved_encoder.table"
    for k in [k1, k2]:
        param = s_dict[k]
        std, mean = torch.std_mean(param)
        param = (param - mean) / std
        bin_in = enumerate_binary_inputs(window=int(math.log2(param.shape[0])))
        print(k)
        print(torch.cat([bin_in.long(), torch.full((bin_in.shape[0], 1), -1), (param > 0).long()], dim=1))
    # print("noninterleaved_encoder.table")
    # print(s_dict["noninterleaved_encoder.table"] - s_dict["noninterleaved_encoder.table"][0])
    # print("interleaved_encoder.table")
    # print(s_dict["interleaved_encoder.table"] - s_dict["interleaved_encoder.table"][0])

block_len=100:lr=0.01:snr=3.0:w=5:init=parity:it=5:max=True:r=False
noninterleaved_encoder.table
tensor([[ 0,  0,  0,  0,  0, -1,  0,  1],
        [ 0,  0,  0,  0,  1, -1,  1,  0],
        [ 0,  0,  0,  1,  0, -1,  1,  0],
        [ 0,  0,  0,  1,  1, -1,  0,  1],
        [ 0,  0,  1,  0,  0, -1,  0,  0],
        [ 0,  0,  1,  0,  1, -1,  1,  0],
        [ 0,  0,  1,  1,  0, -1,  1,  0],
        [ 0,  0,  1,  1,  1, -1,  0,  1],
        [ 0,  1,  0,  0,  0, -1,  1,  1],
        [ 0,  1,  0,  0,  1, -1,  0,  0],
        [ 0,  1,  0,  1,  0, -1,  0,  0],
        [ 0,  1,  0,  1,  1, -1,  1,  1],
        [ 0,  1,  1,  0,  0, -1,  1,  1],
        [ 0,  1,  1,  0,  1, -1,  0,  0],
        [ 0,  1,  1,  1,  0, -1,  0,  0],
        [ 0,  1,  1,  1,  1, -1,  1,  1],
        [ 1,  0,  0,  0,  0, -1,  1,  0],
        [ 1,  0,  0,  0,  1, -1,  0,  1],
        [ 1,  0,  0,  1,  0, -1,  0,  0],
        [ 1,  0,  0,  1,  1, -1,  1,  1],
        [ 1,  0,  1,  0,  0, -1,  1,  0],
        [ 1,  0,  1, 